In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.prune as prune
import numpy as np
from copy import deepcopy
import time
from ResNet import Bottleneck, ResNet, ResNet50
from FLOP_counter import FLOP_counter

In [2]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
net = ResNet50(10).to('cuda')
net.load_state_dict(torch.load('resnet.pt', map_location=torch.device('cuda')))
flops = FLOP_counter(net, (3, 32, 32))

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 64, 64, 1, 1]

In [29]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.49841785430908203
Accuracy on 10,000 test images:  85.42999999999999 %


### Pruning 1

In [13]:
def prune_weight_resnet50(model, percentage = 0.2):
    for name, param in model.named_parameters():
        if 'weight' in name:
            tensor = param.data.cpu().numpy()
            threshold = np.percentile(abs(tensor), percentage*100)
            mask = abs(tensor) > threshold
            param.data = torch.from_numpy(tensor*mask).to('cuda')

In [14]:
# Pruning
pruning = deepcopy(net)
prune_weight_resnet50(pruning, 0.2)
flops = FLOP_counter(pruning).count()
print(flops)

956406763520


In [30]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = pruning(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.5243124961853027
Accuracy on 10,000 test images:  84.14 %


In [15]:
pruning2 = deepcopy(net)
prune_weight_resnet50(pruning2, 0.4)
flops = FLOP_counter(pruning2).count()
print(flops)

956406763520


In [34]:
correct = 0
total = 0

with torch.no_grad():
    start = time.time()
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = pruning2(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('time : ',time.time() - start)
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

time :  0.5123128890991211
Accuracy on 10,000 test images:  62.55 %
